In [ ]:
Оптимизация запросов

В домашнем задании разберите и оптимизируйте SQL-запросы. 
Мы специально не описываем суть запросов словами. 
Вам нужно изучить логику запроса и данные, которые он выводит, а затем понять, 
как получить точно такой же вывод при меньшем числе операций и без нарушения логики вычислений

In [1]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [2]:
HOST = '37.139.42.145'
DBNAME = 'game-analytics'
USER = 'analytics'
PASSWORD = 'BRtTaqYiJyr29WXN'

from sqlalchemy import create_engine
engine = create_engine(f'postgresql://{USER}:{PASSWORD}@{HOST}/{DBNAME}')

group_schema = 'gu_game_analytics_1188'

import pandas as pd
import matplotlib

# ПЕРВОЕ

In [3]:
df = pd.read_sql(f'''
SELECT cast(time as date) as log_dt, 
count(distinct charid) as characters_count
FROM (
    SELECT time, charid, actiondata,
    substring(actiondata, 'ActionName\":\"(\w+)') as actioname
    FROM {group_schema}.pvp_log
    ) t
WHERE actioname = 'QuickMatch'
GROUP BY log_dt
ORDER BY log_dt
''', con=engine)
df

,log_dt,characters_count
0,2018-08-01,1376
1,2018-08-02,1435
2,2018-08-03,1907
3,2018-08-04,1628
4,2018-08-05,1738
5,2018-08-06,1282
6,2018-08-07,1371
7,2018-08-08,1683
8,2018-08-09,1561
9,2018-08-10,1909


Сколько уникальных персонажей учавстовало в QuickMatch по дням. Можно избавиться от подзапроса:

In [4]:
df = pd.read_sql(f'''
SELECT cast(time as date) as log_dt, 
count(distinct charid) as characters_count
FROM {group_schema}.pvp_log
Where substring(actiondata, 'ActionName\":\"(\w+)') = 'QuickMatch'
GROUP BY log_dt
ORDER BY log_dt
''', con=engine)
df

,log_dt,characters_count
0,2018-08-01,1376
1,2018-08-02,1435
2,2018-08-03,1907
3,2018-08-04,1628
4,2018-08-05,1738
5,2018-08-06,1282
6,2018-08-07,1371
7,2018-08-08,1683
8,2018-08-09,1561
9,2018-08-10,1909


# Второе

In [5]:
df = pd.read_sql(f'''
SELECT charid, count(kill) as value
FROM (
SELECT charid, time, cast(substring(actiondata, 'Kill\":(\d+)') as int) as kill
FROM {group_schema}.pvp_log
) t
WHERE time between '2018-08-08 00:00:00' and '2018-08-14 23:59:59'
GROUP BY charid
ORDER BY value DESC
LIMIT 10
''', con=engine)
df

,charid,value
0,5730722,529
1,5787877,404
2,4532857,384
3,3606182,330
4,3175851,322
5,5823469,306
6,3599815,286
7,5798825,284
8,4928583,273
9,5393720,268


Выводит количество убийств за неделю для каждого персонажа, сортирует по убыванию. Также тут можно без подзапроса

In [10]:
df = pd.read_sql(f'''
SELECT DISTINCT charid, 
count(cast(substring(actiondata, 'Kill\":(\d+)') as int)) as value
FROM {group_schema}.pvp_log
WHERE time between '2018-08-08 00:00:00' and '2018-08-14 23:59:59'
GROUP BY charid
ORDER BY value DESC
LIMIT 10
''', con=engine)
df

,charid,value
0,5730722,529
1,5787877,404
2,4532857,384
3,3606182,330
4,3175851,322
5,5823469,306
6,3599815,286
7,5798825,284
8,4928583,273
9,5393720,268


# Третье

In [16]:
df = pd.read_sql(f'''
SELECT charid, rows_count
FROM (
    SELECT charid, rows_count,
    row_number() OVER (ORDER BY rows_count DESC) as rn
    FROM (
        SELECT charid, count(*) as rows_count
        FROM gu_game_analytics_1068.dungeon_log
        GROUP BY charid
        ORDER BY charid
        ) t
    ) t
ORDER BY rn
LIMIT 10
''', con=engine)
df

,charid,rows_count
0,5132780,1908
1,3397847,639
2,4460652,473
3,4673677,407
4,4911710,370
5,5617642,334
6,5706233,302
7,2873645,296
8,5126472,295
9,4749955,293


Выводит топ по количеству походов в данжи для каждого персонажа.  

In [28]:
df = pd.read_sql(f'''
SELECT charid, rows_count
FROM (
    SELECT charid, count(*) as rows_count,
    row_number() OVER (ORDER BY count(*) DESC) as rn
    FROM gu_game_analytics_1068.dungeon_log
    GROUP BY charid
    ORDER BY rn
    ) as t
LIMIT 10
    ''', con=engine)
df

,charid,rows_count
0,5132780,1908
1,3397847,639
2,4460652,473
3,4673677,407
4,4911710,370
5,5617642,334
6,5706233,302
7,2873645,296
8,5126472,295
9,4749955,293


# Четвертое

In [8]:
df = pd.read_sql(f'''
SELECT charclass, count(*) as dungeon_characters_cnt
FROM {group_schema}.characters
WHERE id in (
    SELECT charid
    FROM {group_schema}.dungeon_log
    ORDER BY charid
    )
AND charlevel >= 50
GROUP BY charclass
''', con=engine)
df

,charclass,dungeon_characters_cnt
0,Archer,1075
1,Assasin,876
2,Bard,778
3,Mage,2341
4,Warrior,1927


Выводит количество персонажей по классам, которые ходили в данж, имея уровень больше 50ого. Нет идей как упростить код(

# Пятое

In [30]:
df = pd.read_sql(f'''
SELECT 'dungeon_log' as log_name, count(*) as rows_count
FROM {group_schema}.dungeon_log
WHERE charid in (
    SELECT charid
    FROM {group_schema}.pvp_log
    GROUP BY charid
    ORDER BY count(*) DESC
    LIMIT 50
    )
UNION ALL
SELECT 'pvp_log', count(*) as rows_count
FROM {group_schema}.pvp_log
WHERE charid in (
    SELECT charid
    FROM {group_schema}.pvp_log
    GROUP BY charid
    ORDER BY count(*) DESC
    LIMIT 50
    )
''', con=engine)
df

,log_name,rows_count
0,dungeon_log,2143
1,pvp_log,40153


In [ ]:
сколько 50 самых активных игроков в пвп сходили раз в данжи 
и сколько 50 самых активных проходчиков данжей - сыграли в пвп битвы.
Жесть, чуть голову не сломал) но вроде именно это и выводит.